In [21]:
import numpy as np
import pandas as pd
seed = 32
import warnings
warnings.filterwarnings('ignore')
target =  'APP'

# Read ELHMA data 

In [22]:
"""def read_and_reformate_data(path):
    original_data = pd.read_csv(path)
     ### derive PLV_Totale and drop PLV of all stations
    original_data['PLV'] = original_data.iloc[:,4:24].sum(axis=1)    
    original_data.drop(original_data.iloc[:,4:23], axis=1, inplace=True)
    original_data.set_index('DATE', inplace=True)
    original_data.to_csv('hma_full_data.csv',index=False, sep=',')
    return original_data"""

"def read_and_reformate_data(path):\n    original_data = pd.read_csv(path)\n     ### derive PLV_Totale and drop PLV of all stations\n    original_data['PLV'] = original_data.iloc[:,4:24].sum(axis=1)    \n    original_data.drop(original_data.iloc[:,4:23], axis=1, inplace=True)\n    original_data.set_index('DATE', inplace=True)\n    original_data.to_csv('hma_full_data.csv',index=False, sep=',')\n    return original_data"

In [23]:
"""elhma_data = read_and_reformate_data('full_dataset.csv')"""

"elhma_data = read_and_reformate_data('full_dataset.csv')"

# Read Kelibia climatic data

In [24]:
import datetime
def read_and_reformate_climatic_data(path):
    df = pd.read_excel(path, sheet_name='KELIBIA')
    df = df.drop(['NUM_STA','NOM_STA','Unnamed: 7'],axis=1)
    df.rename(columns={'DAT': 'DATE', 'Temp. Minimale °C' : 'Temp_min', 'Temp. Maximale °C':'Temp_max',
                        'Précipitation toat (mm)':'PLV', 'Evaporation Pch (mm)':'EVA'},inplace=True)
    df['DATE'] = pd.DatetimeIndex(df['DATE'])   
    df.set_index('DATE', inplace=True)
    return df

In [25]:
kelibia_climatic_data = read_and_reformate_climatic_data('KELIBIA.xlsx')

# Read Kelibia inflow data

In [26]:
# Load inflow target
def read_and_reformate_inflow(file):
    #convert string french months to numeric values
    df = pd.read_excel(file, sheet_name='Sheet1')
    df.rename(columns={'apport':'APP'},inplace=True)
    dates = df.pop('date')
    dates = dates.str.split(' ')
    months_names = ['Janvier','Février','Mars','Avril','Mai','Juin','Juillet','Aôut','Septembre','Octobre','Novembre','Décembre']
    months_values = ['01/01','02/01','03/01','04/01','05/01','06/01','07/01','08/01','09/01','10/01','11/01','12/01']
    data_date_format = []
    for date in dates:
        index = months_names.index(date[0])
        data_date_format.append(months_values[index]+'/'+date[1])
    df['date']= pd.DatetimeIndex(data_date_format)  
    df.set_index('date', inplace=True)
    return df

In [27]:
kelibia_inflow_data = read_and_reformate_inflow('apport.xlsx')

# Concatenate kelibia climatic data & inflow

In [28]:
from sklearn.model_selection import train_test_split
def concatunate_inflow(data_cleaned, inflow): 
    data = data_cleaned.copy()
    data['APP'] = np.nan
    for year in sorted(pd.unique(pd.DatetimeIndex(inflow.index).year)):
        for month in range(1,13):
            index_app = list(inflow[np.logical_and(year==pd.DatetimeIndex(inflow.index).year, 
                                                   month==pd.DatetimeIndex(inflow.index).month)].index)
            index_data = list(data[np.logical_and(year==pd.DatetimeIndex(data.index).year, 
                                               month==pd.DatetimeIndex(data.index).month)].index)
            if len(index_data)>0 and len(index_app)>0:
                data.loc[index_data, 'APP'] = inflow.loc[index_app, 'APP'].values[0]   
    data.dropna(axis=0,inplace=True)
    return data

kelibia_data = concatunate_inflow(kelibia_climatic_data, kelibia_inflow_data)

# Create statistical data for Kelibia data

In [32]:
def create_statistic_data(full_data):
    new_data = pd.DataFrame()
    #For each year
    full_data['DATE'] = full_data.index
    for year in pd.unique(pd.DatetimeIndex(full_data['DATE']).year):
        local_year_df = full_data[pd.DatetimeIndex(full_data['DATE']).year == year]
        #for each month
        for month in pd.unique(pd.DatetimeIndex(local_year_df['DATE']).month):
            #Get all samples for each month
            KELIBIA = full_data[(pd.DatetimeIndex(full_data['DATE']).month== month) & (pd.DatetimeIndex(full_data['DATE']).year ==year)]
            date = KELIBIA[['DATE']]
            # get min, max, mean, std, 25%, 50% and 75% for each column
            KELIBIA = KELIBIA.describe().drop(['count'],axis=0)
            #convert the DF from N*M dimonssion to 1*(M*N)  dimonssion
            KELIBIA = KELIBIA.unstack().to_frame().T
            KELIBIA.columns = ['_'.join(column) for column in KELIBIA.columns]
            #set the first date to meet as index 
            KELIBIA['DATE'] = date.iloc[0][0]
            KELIBIA = KELIBIA.set_index(KELIBIA['DATE'])

            new_data = new_data.append(KELIBIA)
    new_data = new_data.drop(['DATE','APP_std','APP_min','APP_25%','APP_50%','APP_75%','APP_max'],axis=1)
    new_data.rename(columns={'APP_mean':'APP'},inplace=True)

    return new_data  

In [33]:
#kelibia_statistical_data = create_statistic_data(kelibia_data)

In [34]:
def get_percent_fo_missing_values(full_data):
    data = full_data
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (100*data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data[missing_data.Percent>0]

In [35]:
#get_percent_fo_missing_values(kelibia_statistical_data)
get_percent_fo_missing_values(kelibia_data)

,Total,Percent


In [36]:
"""get_percent_fo_missing_values(elhma_data)"""

'get_percent_fo_missing_values(elhma_data)'

# Data preprocessing

In [37]:
def preprocessing(original_data):
    clean_data = original_data.copy()
    clean_data.drop_duplicates(keep='first', inplace=True)
    ### fill missing data 
    clean_data.interpolate(inplace=True)
    clean_data.fillna(method='bfill',inplace=True)    
    return clean_data

In [38]:
"""elhma_data = preprocessing(elhma_data)"""

'elhma_data = preprocessing(elhma_data)'

In [39]:
#kelibia_statistical_data = preprocessing(kelibia_statistical_data)
kelibia_data = preprocessing(kelibia_data)

# Split & Scaling & Transformation

In [41]:
from sklearn.model_selection import train_test_split
def split_data_into_train_test_sets(ts_features_targets):
    X = ts_features_targets.drop('APP', axis=1)
    y = ts_features_targets[['APP']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=seed, shuffle=True)
    return X_train, X_test, y_train, y_test

In [42]:
# scaling and transform data
from sklearn.preprocessing import PowerTransformer, StandardScaler
def data_scaling_transform(X_train, X_test, y_train, y_test):
    ### transform input variables
    scaler = StandardScaler().fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    ### transform target variables
    transformer = PowerTransformer().fit(y_train)
    y_train = transformer.transform(y_train)
    y_test = transformer.transform(y_test)     
    return X_train, X_test, y_train, y_test

In [43]:
X_train, X_test, y_train, y_test = split_data_into_train_test_sets(kelibia_statistical_data)
X_train, X_test, y_train, y_test = data_scaling_transform(X_train, X_test, y_train, y_test)

NameError: name 'kelibia_statistical_data' is not defined

In [ ]:
"""X_train, X_test, y_train, y_test = split_data_into_train_test_sets(ts_features_targets)
X_train, X_test, y_train, y_test = data_scaling_transform(X_train, X_test, y_train, y_test)"""

In [46]:
#kelibia_statistical_data.to_csv('kelibia_statistical_data.csv',index=False, sep=',')
kelibia_data.to_csv('kelibia_data.csv', sep=',')